# Fake News Detection 

## importing the libraries : 

In [86]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Loading the Data : 

the fake news data is in the file 'fake.csv'

In [87]:
fake_news = pd.read_csv('data/fake.csv')
fake_news['status'] = 1
fake_news.head()

,title,text,subject,date,status
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


the real news in the file 'true.csv'

In [88]:
real_news = pd.read_csv('data/true.csv')
real_news['status'] = 0
real_news.head()

,title,text,subject,date,status
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


Concatenating Real and Fake News DataFrames

In [89]:
data = pd.concat([real_news, fake_news], ignore_index=True)
data.head()

,title,text,subject,date,status
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [90]:
data.shape

(44898, 5)

## Preprocessing : 
Data preprocessing is an important step in the data mining process. It refers to the cleaning, transforming, and integrating of data in order to make it ready for analysis. The goal of data preprocessing is to improve the quality of the data and to make it more suitable for the specific data mining task.

In [91]:
data.isnull().sum()

title      0
text       0
subject    0
date       0
status     0
dtype: int64

## Stemming : 
Stemming is a method in text processing that eliminates prefixes and suffixes from words, transforming them into their fundamental or root form, The main objective of stemming is to streamline and standardize words, enhancing the effectiveness of the natural language processing tasks.

In [92]:
# initialize an instance of the Porter stemming
ps = PorterStemmer() 
def stemming(content):
    # remove any characters that are not alphabetic 
    stemmed = re.sub('[^a-zA-Z]',' ',content)
    stemmed = stemmed.lower()
    stemmed = stemmed.split()
    """ 
        applies stemming using the Porter stemming algorithm (ps.stem(word)), and checks if the stemmed word is 
        not in NLTK's list of English stopwords. Stopwords are common words that are often filtered out during text 
        processing because they typically do not carry much meaning. 
    """
    stemmed = [ps.stem(word) for word in stemmed if not word in stopwords.words('english')]
    stemmed = ' '.join(stemmed)
    return stemmed

we will try to detect the fake news based on the title

In [93]:
data['content'] = data['title'] 
data['content']

0        As U.S. budget fight looms, Republicans flip t...
1        U.S. military to accept transgender recruits o...
2        Senior U.S. Republican senator: 'Let Mr. Muell...
3        FBI Russia probe helped by Australian diplomat...
4        Trump wants Postal Service to charge 'much mor...
                               ...                        
44893    McPain: John McCain Furious That Iran Treated ...
44894    JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
44895    Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
44896    How to Blow $700 Million: Al Jazeera America F...
44897    10 U.S. Navy Sailors Held by Iranian Military ...
Name: content, Length: 44898, dtype: object

In [94]:
data['content'] = data['content'].apply(stemming)
data['content']

0        u budget fight loom republican flip fiscal script
1        u militari accept transgend recruit monday pen...
2             senior u republican senat let mr mueller job
3        fbi russia probe help australian diplomat tip nyt
4        trump want postal servic charg much amazon shi...
                               ...                        
44893    mcpain john mccain furiou iran treat us sailor...
44894    justic yahoo settl e mail privaci class action...
44895    sunnistan us alli safe zone plan take territor...
44896      blow million al jazeera america final call quit
44897    u navi sailor held iranian militari sign neoco...
Name: content, Length: 44898, dtype: object

In [95]:
Y = data['status'].values
X = data['content'].values

## converting the textual data to numerical data : 

In [96]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [97]:
print(X)

  (0, 10256)	0.4818987064386032
  (0, 9654)	0.22293424208165316
  (0, 6846)	0.39492067407001413
  (0, 4400)	0.4235483019813407
  (0, 4336)	0.43799147037450814
  (0, 4278)	0.2950519420321273
  (0, 1562)	0.3222208573375334
  (1, 11921)	0.3885513402289849
  (1, 9470)	0.4664844508728347
  (1, 8598)	0.4005461104074412
  (1, 7563)	0.44625466974222106
  (1, 7402)	0.3279450434773877
  (1, 62)	0.4053291807031351
  (2, 10345)	0.4078026132599872
  (2, 10340)	0.26203111154891795
  (2, 9654)	0.255164221251294
  (2, 7676)	0.4235484575672548
  (2, 7665)	0.49944198217358804
  (2, 6681)	0.3846685071175455
  (2, 6175)	0.35088815249144584
  (3, 11786)	0.4592487727510474
  (3, 9993)	0.24717627310248227
  (3, 9055)	0.29977050093679036
  (3, 8050)	0.4114336528025053
  (3, 5384)	0.3058534768546792
  :	:
  (44895, 11478)	0.20603393353637509
  (44895, 11448)	0.2162746619702823
  (44895, 11328)	0.43022545876064566
  (44895, 10031)	0.30492863621960886
  (44895, 8776)	0.19462776085820013
  (44895, 7973)	0.2897402

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [99]:
X_train.shape

(35918, 13206)

## Training the Model: Logistic Regression :

In [100]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [101]:
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

0.9604933459546745


In [102]:
testing_y_pred = model.predict(X_test)
print(accuracy_score(testing_y_pred,Y_test))

0.9415367483296214


In [103]:
input_data = X_test[2765]
prediction = model.predict(input_data)

In [104]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

The News is Fake
